In [ ]:
# Install necessary packages (if not already installed)
# These packages allow for database connections, data manipulation, and visualization

%pip install pyodbc
%pip install pymssql
%pip install pandas
%pip install sqlalchemy
%pip install urllib
%pip install matplotlib
%pip install seaborn
!pip install pandas matplotlib seaborn sqlalchemy

In [ ]:
# Importing essential libraries
import pandas as pd
from sqlalchemy import create_engine, text
import pyodbc
import pymssql
import matplotlib.pyplot as plt
import seaborn as sns
import glob
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.cluster import KMeans, DBSCAN
from sklearn.mixture import GaussianMixture
from sklearn.metrics import silhouette_score , accuracy_score
from sklearn.ensemble import RandomForestClassifier
import joblib
import pickle
import mlflow
import mlflow.sklearn
from mlflow import MlflowClient
from mlflow.tracking import MlflowClient

In [ ]:
## Data Werehouse
# Step 1: Set up the connection to the OLTP (Operational) Database
# Using pyodbc to connect to the OLTP database
conn = pyodbc.connect(
    'Driver={SQL Server};'
    'Server=DESKTOP-DB1EO4N;'  # Update with your actual server name
    'Database=customers;'       # Name of your OLTP database
    'Trusted_Connection=yes;'   # Using Windows Authentication
)

In [ ]:
# Step 2: Test the connection with a simple query
# Creating a cursor to execute a simple query to validate the connection
# Create a cursor
cursor = conn.cursor()
# Optionally, run a simple query to test the connection
try:
    cursor.execute("SELECT 1")  # Test query
    row = cursor.fetchone()
    print("Query result:", row)  # Should return (1,)
except Exception as e:
    print("Error executing query:", e)
finally:
    cursor.close()  # Always close the cursor after usage
    conn.close()    # Close the connection

In [ ]:
# Step 3: Create engines for both OLTP and DWH databases
# These engines will be used to fetch data and insert it into the DWH
oltp_engine = create_engine('mssql+pyodbc://@localhost/customers?driver=ODBC+Driver+17+for+SQL+Server&Trusted_Connection=yes')
dwh_engine = create_engine('mssql+pyodbc://@localhost/DWH_DB?driver=ODBC+Driver+17+for+SQL+Server&Trusted_Connection=yes')


In [ ]:
# Step 4: Extract data from OLTP database into pandas DataFrames
# Fetching the tables from the OLTP system (customers, products, staff, stores, etc.)
customers_df = pd.read_sql("SELECT * FROM customer", con=oltp_engine)
products_df = pd.read_sql("SELECT * FROM products", con=oltp_engine)
staff_df = pd.read_sql("SELECT * FROM staffs", con=oltp_engine)
stores_df = pd.read_sql("SELECT * FROM store", con=oltp_engine)
category_df = pd.read_sql("SELECT * FROM category", con=oltp_engine)
brand_df = pd.read_sql("SELECT * FROM brand", con=oltp_engine)
stocks_df = pd.read_sql("SELECT * FROM stocks", con=oltp_engine)
orders_df = pd.read_sql("SELECT * FROM orders", con=oltp_engine)


In [ ]:
# Step 5: Perform basic transformations on the data
# Handling missing values by filling NaNs with a placeholder in email columns
customers_df['email'].fillna('noemail@example.com')
staff_df['email'].fillna('noemail@example.com')

In [ ]:
# Step 6: Select relevant columns to create dimension tables
# Reducing the dataframes to include only the necessary columns for each dimension table
dim_customers_df = customers_df[['customer_id', 'first_name', 'last_name', 'email', 'street', 'city', 'state', 'zip_code']]
dim_products_df = products_df[['product_id', 'brand_id', 'category_id', 'product_name', 'model_year', 'list_price', 'list_price_percentage']]
dim_staff_df = staff_df[['staff_id', 'first_name', 'last_name', 'email', 'store_id', 'manager_id', 'active']]
dim_stores_df = stores_df[['store_id', 'store_name', 'phone', 'email', 'street', 'city', 'state', 'zip_code']]
dim_category_df = category_df[['category_id', 'category_name']]
dim_brand_df = brand_df[['brand_id', 'brand_name']]
dim_stocks_df = stocks_df[['store_id', 'product_id', 'quantity']]
dim_orders_df = orders_df[['order_id', 'staff_id', 'customer_id', 'store_id', 'order_status', 'order_date', 'required_date', 'shipped_date']]


In [ ]:
# Step 7: Load the transformed data into the Data Warehouse (DWH)
# Inserting the data into the respective dimension tables in the DWH
dim_category_df.to_sql('DimCategory', con=dwh_engine, if_exists='append', index=False)
dim_brand_df.to_sql('DimBrand', con=dwh_engine, if_exists='append', index=False)
dim_stores_df.to_sql('DimStore', con=dwh_engine, if_exists='append', index=False)
dim_customers_df.to_sql('DimCustomer', con=dwh_engine, if_exists='append', index=False)
dim_products_df.to_sql('DimProduct', con=dwh_engine, if_exists='append', index=False)
dim_staff_df.to_sql('DimStaff', con=dwh_engine, if_exists='append', index=False)
dim_stocks_df.to_sql('DimStocks', con=dwh_engine, if_exists='append', index=False)
dim_orders_df.to_sql('DimOrders', con=dwh_engine, if_exists='append', index=False)

print("Dimension tables loaded successfully into the Data Warehouse.")
stocks_df = pd.read_sql("SELECT * FROM stocks", con=oltp_engine)
dim_stocks_df = stocks_df[['store_id', 'product_id', 'quantity']]
dim_stocks_df.to_sql('Dimstocks', con=dwh_engine, if_exists='append', index=False)

In [ ]:
# Step 8: Handle duplicates (if applicable)
# Example: Check for duplicate customer_id values in the DimCustomer table
duplicate_check = dim_customers_df[dim_customers_df.duplicated(subset=['customer_id'], keep=False)]

# If no duplicates, print confirmation; otherwise, print the number of duplicates found
if duplicate_check.empty:
    print("No duplicate customer_id found!")
else:
    print(f"Found {len(duplicate_check)} duplicates.")



In [ ]:
## Analsis with Python
# Step 1: Specify the folder path where your CSV files are stored
folder_path = "/content/datase

In [ ]:
# Step 2: Use glob to find all CSV files in the folder and store their paths
csv_files = glob.glob(folder_path + "/*.csv")

In [ ]:
# Step 3: Load each CSV file into a DataFrame and store them in a dictionary
dataframes = {file.split('/')[-1]: pd.read_csv(file) for file in csv_files}


In [ ]:
# Step 4: Print the first few rows of each loaded DataFrame to verify the data
for name, df in dataframes.items():
    print(f"Table: {name}")
    print(df.head(), "\n")

In [ ]:
# Step 5: Print the column names of each DataFrame for a quick overview
for name, df in dataframes.items():
    print(f"{name} columns: {df.columns}\n")

In [ ]:
# Step 6: Load specific DataFrames, assuming two tables: 'customers.csv' and 'orders.csv'
customers = dataframes['customers.csv']
orders = dataframes['orders.csv']

In [ ]:
# Step 7: Merge 'orders' with 'customers' on 'customer_id' to associate orders with customer details
merged_df = pd.merge(orders, customers, on='customer_id', how='inner')

In [ ]:
# Step 8: Display the first few rows of the merged DataFrame to check the result
print(merged_df.head())

In [ ]:
# Step 9: Load more CSV files as DataFrames for further analysis
products = pd.read_csv('/content/dataset/products.csv')
brands = pd.read_csv('/content/dataset/brands.csv')
categories = pd.read_csv('/content/dataset/categories.csv')
order_items = pd.read_csv('/content/dataset/order_items.csv')
orders = pd.read_csv('/content/dataset/orders.csv')
customers = pd.read_csv('/content/dataset/customers.csv')
stores = pd.read_csv('/content/dataset/stores.csv')
staffs = pd.read_csv('/content/dataset/staffs.csv')
stocks = pd.read_csv('/content/dataset/stocks.csv')


In [ ]:
# Step 10: Check the first few rows of the products and brands DataFrames
print(products.head())  # Products DataFrame
print(brands.head())    # Brands DataFrame

In [ ]:
# Step 11: Merge 'products' with 'brands' using 'brand_id' to bring brand information to products
merged_products_brands = pd.merge(products, brands, on='brand_id', how='inner')

In [ ]:
# Step 12: Display the merged products-brands DataFrame
print(merged_products_brands.head())

In [ ]:
# Step 13: Merge 'products' with 'brands', then merge the result with 'categories'
products_brands = pd.merge(products, brands, on='brand_id', how='inner')
products_full = pd.merge(products_brands, categories, on='category_id', how='inner')


In [ ]:
# Step 14: Merge 'orders' with 'customers' for customer-related analysis
orders_customers = pd.merge(orders, customers, on='customer_id', how='inner')

In [ ]:
# Step 15: Merge 'order_items' with 'products_full' to associate items with their product and brand info
order_items_products = pd.merge(order_items, products_full, on='product_id', how='inner')


In [ ]:
# Step 16: Merge 'orders_customers' with 'stores' and 'staffs' to add store and staff details
orders_full = pd.merge(orders_customers, stores, on='store_id', how='inner')
orders_full = pd.merge(orders_full, staffs, on='staff_id', how='inner')


In [ ]:
# Step 17: Print the final merged DataFrame for 'order_items' with product, store, and customer details
print(order_items_products.head())  # Check final order_items merged DataFrame


# Merge products with brands and categories
products_full = pd.merge(products, brands, on='brand_id', how='inner')
products_full = pd.merge(products_full, categories, on='category_id', how='inner')

# Merge order_items with products_full
order_items_products = pd.merge(order_items, products_full, on='product_id', how='inner')

# Merge orders with customers and stores
orders_customers = pd.merge(orders, customers, on='customer_id', how='inner')
orders_full = pd.merge(orders_customers, stores, on='store_id', how='inner')


In [ ]:
# Step 18: Merge 'order_items_products' with 'orders_full' to obtain the final sales data
sales_data = pd.merge(order_items_products, orders_full, on='order_id', how='inner')


In [ ]:
# Step 19: Verify the final merged sales data by printing the first few rows
print(sales_data.head())


In [ ]:
# Step 20: Print the column names of the sales data to see all available fields
print(sales_data.columns)


In [ ]:
# Step 21: Group sales data by 'product_name' and calculate the total quantity sold for each product
top_products = sales_data.groupby('product_name')['quantity'].sum().reset_index()


In [ ]:
# Step 22: Sort the top products by quantity sold in descending order and select the top 10
top_products = top_products.sort_values(by='quantity', ascending=False).head(10)


In [ ]:
# Step 23: Print the top 10 best-selling products
print("Top 10 Best-Selling Products:")
print(top_products)


In [ ]:
# Step 24: Plot a bar chart for the top 10 best-selling products
plt.figure(figsize=(12, 6))
sns.barplot(data=top_products, x='quantity', y='product_name', palette='Blues_r')
plt.title('Top 10 Best-Selling Products')
plt.xlabel('Quantity Sold')
plt.ylabel('Product Name')
plt.show()

In [ ]:
# Step 25: Calculate total spending per order item (quantity * (list_price - discount))
sales_data['total_spending'] = sales_data['quantity'] * (sales_data['list_price_x'] - sales_data['discount'])


In [ ]:
# Step 26: Group the data by customer and calculate total spending per customer
top_customers = sales_data.groupby(['customer_id', 'first_name', 'last_name'])['total_spending'].sum().reset_index()


In [ ]:
# Step 27: Sort the top customers by total spending and select the top 10
top_customers = top_customers.sort_values(by='total_spending', ascending=False).head(10)


In [ ]:
# Step 28: Print the top 10 customers by total spending
print("Top 10 Customers by Total Spending:")
print(top_customers)


In [ ]:
# Step 29: Plot a bar chart for the top 10 customers by total spending
plt.figure(figsize=(12, 6))
sns.barplot(data=top_customers, x='total_spending', y='last_name', palette='Greens_r')
plt.title('Top 10 Customers by Total Spending')
plt.xlabel('Total Spending')
plt.ylabel('Customer (Last Name)')
plt.show()

In [ ]:
# Step 30: Group sales data by 'store_name' and calculate total revenue per store
store_revenue = sales_data.groupby('store_name')['total_spending'].sum().reset_index()


In [ ]:
# Step 31: Sort stores by total revenue in descending order and print the result
store_revenue = store_revenue.sort_values(by='total_spending', ascending=False)
print("Revenue by Store:")
print(store_revenue)

In [ ]:
# Step 32: Group sales data by 'category_name' to calculate total revenue by product category
category_revenue = sales_data.groupby('category_name')['total_spending'].sum().reset_index()
category_revenue = category_revenue.sort_values(by='total_spending', ascending=False)
print("Revenue by Product Category:")
print(category_revenue)

In [ ]:
# Step 33: Group sales data by 'brand_name' to calculate total revenue for each brand
brand_performance = sales_data.groupby('brand_name')['total_spending'].sum().reset_index()
brand_performance = brand_performance.sort_values(by='total_spending', ascending=False)
print("Brand Performance by Revenue:")
print(brand_performance)

In [ ]:
# Step 34: Convert 'order_date' to datetime and extract the month for monthly sales analysis
sales_data['order_date'] = pd.to_datetime(sales_data['order_date'])
sales_data['month'] = sales_data['order_date'].dt.to_period('M')


In [ ]:
# Step 35: Group sales data by 'month','store_name','product_name','state_x' and calculate total spending per month

# Group sales data by 'month' and calculate the total spending for each month
monthly_sales = sales_data.groupby('month')['total_spending'].sum().reset_index()

# Print the monthly sales trend to inspect how sales change over time
print("Monthly Sales Trend:")
print(monthly_sales)

# Group sales data by 'store_name' to calculate the total quantity of items sold by each store
popular_stores = sales_data.groupby('store_name')['quantity'].sum().reset_index()

# Sort the stores by quantity sold in descending order and get the top 10 performing stores
popular_stores = popular_stores.sort_values(by='quantity', ascending=False).head(10)

# Print the top 10 stores by the total quantity of products sold
print("Top 10 Stores by Quantity Sold:")
print(popular_stores)

# Group sales data by 'product_name' and calculate the average discount for each product
avg_discount = sales_data.groupby('product_name')['discount'].mean().reset_index()

# Sort products by the average discount in descending order to see which products get the highest discounts
avg_discount = avg_discount.sort_values(by='discount', ascending=False)

# Print the products with the highest average discount
print("Average Discount per Product:")
print(avg_discount)

# Group sales data by 'state_x' (representing the state of the customers) and count the unique customers in each state
customers_by_state = sales_data.groupby('state_x')['customer_id'].nunique().reset_index()

# Sort the states by the number of unique customers in descending order
customers_by_state = customers_by_state.sort_values(by='customer_id', ascending=False)

# Print the number of unique customers by state
print("Number of Unique Customers by State:")
print(customers_by_state)

# Group sales data by staff (using 'staff_id', 'first_name', and 'last_name') to calculate the total revenue generated by each staff member
staff_revenue = sales_data.groupby(['staff_id', 'first_name', 'last_name'])['total_spending'].sum().reset_index()

# Sort staff members by total revenue in descending order to see the top-performing staff
staff_revenue = staff_revenue.sort_values(by='total_spending', ascending=False)

# Print the revenue generated by each staff member
print("Revenue by Staff Member:")
print(staff_revenue)


In [ ]:
# Step 36: Plot a line chart
# Plotting Store Revenue
plt.figure(figsize=(12, 6))
sns.barplot(data=store_revenue, x='total_spending', y='store_name', palette='Oranges_r')
plt.title('Revenue by Store')
plt.xlabel('Total Revenue')
plt.ylabel('Store Name')
plt.show()


# Additional Step: Plotting Revenue by Product Category
plt.figure(figsize=(8, 8))
plt.pie(category_revenue['total_spending'], labels=category_revenue['category_name'],
        autopct='%1.1f%%', startangle=140, colors=sns.color_palette('pastel'))
plt.title('Revenue by Product Category')
plt.show()


# Additional Step: Ensure 'month' column is in datetime format
monthly_sales['month'] = pd.to_datetime(monthly_sales['month'].astype(str), format='%Y-%m')

# Additional Step: Plotting Monthly Sales Trend
plt.figure(figsize=(12, 6))
sns.lineplot(data=monthly_sales, x='month', y='total_spending', marker='o', color='purple')
plt.title('Monthly Sales Trend')
plt.xlabel('Month')
plt.ylabel('Total Spending')
plt.xticks(rotation=45)
plt.show()



# Additional Step: Plotting Brand Performance by Revenue
plt.figure(figsize=(12, 6))
sns.barplot(data=brand_performance, x='total_spending', y='brand_name', palette='coolwarm')
plt.title('Brand Performance by Revenue')
plt.xlabel('Total Revenue')
plt.ylabel('Brand Name')
plt.show()


# Additional Step: Plotting Top 10 Stores by Quantity Sold
plt.figure(figsize=(12, 6))
sns.barplot(data=popular_stores, x='quantity', y='store_name', palette='magma')
plt.title('Top 10 Stores by Quantity Sold')
plt.xlabel('Total Quantity Sold')
plt.ylabel('Store Name')
plt.show()

# Additional Step: Plotting Number of Unique Customers by State
plt.figure(figsize=(12, 6))
sns.barplot(data=customers_by_state, x='customer_id', y='state_x', palette='viridis')
plt.title('Number of Unique Customers by State')
plt.xlabel('Number of Customers')
plt.ylabel('State')
plt.show()




In [ ]:
##Cluster Model
##Load and Preprocess the Date
# Load the sales and customer datasets from CSV files

sales_data = pd.read_csv('sales_data.csv')
customer_data = pd.read_csv('customer_data.csv')

# Check the first few rows of both datasets to inspect their structure
print(customer_data.head())
print(sales_data.head())

In [ ]:

# Preprocessing customer data

# Fill missing values in the 'age' column with the median of the column
customer_data['age'].fillna(customer_data['age'].median(), inplace=True)

# Initialize LabelEncoder to convert categorical variables into numeric format
label_encoder = LabelEncoder()

# Apply label encoding on categorical columns in the customer_data
customer_data['customer_id'] = label_encoder.fit_transform(customer_data['customer_id'])
customer_data['gender'] = label_encoder.fit_transform(customer_data['gender'])
customer_data['payment_method'] = label_encoder.fit_transform(customer_data['payment_method'])

# Preprocessing sales data

# Convert 'invoice_date' column to datetime format to allow for date-based analysis
sales_data['invoice_date'] = pd.to_datetime(sales_data['invoice_date'], format='%d-%m-%Y')

# Extract year, month, and day from 'invoice_date' for feature creation
sales_data['invoice_year'] = sales_data['invoice_date'].dt.year
sales_data['invoice_month'] = sales_data['invoice_date'].dt.month
sales_data['invoice_day'] = sales_data['invoice_date'].dt.day

# Apply label encoding on categorical columns in sales_data
sales_data['invoice_no'] = label_encoder.fit_transform(sales_data['invoice_no'])
sales_data['customer_id'] = label_encoder.fit_transform(sales_data['customer_id'])
sales_data['category'] = label_encoder.fit_transform(sales_data['category'])
sales_data['shopping_mall'] = label_encoder.fit_transform(sales_data['shopping_mall'])

# Drop the 'invoice_date' column since we have already extracted relevant features from it
sales_data = sales_data.drop(columns=['invoice_date'])

In [ ]:
# Initialize the scaler
scaler = StandardScaler()

# Scale the customer and sales data using the standard scaler
customer_data_scaled = scaler.fit_transform(customer_data)
sales_data_scaled = scaler.fit_transform(sales_data)

In [ ]:


# Apply K-Means clustering with 2 clusters on the scaled sales data and customer data
kmeans_sales = KMeans(n_clusters=2, random_state=42).fit(sales_data_scaled)
kmeans_customer = KMeans(n_clusters=2, random_state=42).fit(customer_data_scaled)

# Apply DBSCAN clustering on scaled sales and customer data
dbscan_sales = DBSCAN(eps=0.5, min_samples=5).fit(sales_data_scaled)
dbscan_customer = DBSCAN(eps=0.5, min_samples=5).fit(customer_data_scaled)

# Apply Gaussian Mixture Models (GMM) clustering with 3 components
gmm_sales = GaussianMixture(n_components=3, random_state=42).fit(sales_data_scaled)
gmm_customer = GaussianMixture(n_components=3, random_state=42).fit(customer_data_scaled)

# Save the clustering models using joblib for future use
joblib.dump(kmeans_sales, 'kmeans_sales.pkl')
joblib.dump(kmeans_customer, 'kmeans_customer.pkl')
joblib.dump(dbscan_sales, 'dbscan_sales.pkl')
joblib.dump(dbscan_customer, 'dbscan_customer.pkl')
joblib.dump(gmm_sales, 'gmm_sales.pkl')
joblib.dump(gmm_customer, 'gmm_customer.pkl')

# Print message to confirm model saving
print("Models saved successfully.")

In [ ]:

# Apply K-Means clustering with 2 clusters on the scaled sales data and customer data
kmeans_sales = KMeans(n_clusters=2, random_state=42).fit(sales_data_scaled)
kmeans_customer = KMeans(n_clusters=2, random_state=42).fit(customer_data_scaled)

# Apply DBSCAN clustering on scaled sales and customer data
dbscan_sales = DBSCAN(eps=0.5, min_samples=5).fit(sales_data_scaled)
dbscan_customer = DBSCAN(eps=0.5, min_samples=5).fit(customer_data_scaled)

# Apply Gaussian Mixture Models (GMM) clustering with 3 components
gmm_sales = GaussianMixture(n_components=3, random_state=42).fit(sales_data_scaled)
gmm_customer = GaussianMixture(n_components=3, random_state=42).fit(customer_data_scaled)

# Save K-Means, DBSCAN, and GMM models using pickle
with open('kmeans_sales.pkl', 'wb') as f:
    pickle.dump(kmeans_sales, f)

with open('kmeans_customer.pkl', 'wb') as f:
    pickle.dump(kmeans_customer, f)

with open('dbscan_sales.pkl', 'wb') as f:
    pickle.dump(dbscan_sales, f)

with open('dbscan_customer.pkl', 'wb') as f:
    pickle.dump(dbscan_customer, f)

with open('gmm_sales.pkl', 'wb') as f:
    pickle.dump(gmm_sales, f)

with open('gmm_customer.pkl', 'wb') as f:
    pickle.dump(gmm_customer, f)

# Confirm that the models are saved successfully
print("Models saved successfully.")


In [ ]:


# Calculate silhouette scores for K-Means clustering results
kmeans_sales_score = silhouette_score(sales_data_scaled, kmeans_sales.labels_)
kmeans_customer_score = silhouette_score(customer_data_scaled, kmeans_customer.labels_)

# Calculate silhouette score for DBSCAN results if not all points are classified as noise
if len(set(dbscan_sales.labels_)) > 1:
    dbscan_sales_score = silhouette_score(sales_data_scaled, dbscan_sales.labels_)
else:
    dbscan_sales_score = 'Not applicable (too many points classified as noise)'

if len(set(dbscan_customer.labels_)) > 1:
    dbscan_customer_score = silhouette_score(customer_data_scaled, dbscan_customer.labels_)
else:
    dbscan_customer_score = 'Not applicable (too many points classified as noise)'

# Calculate silhouette score for Gaussian Mixture Model clustering results
gmm_sales_score = silhouette_score(sales_data_scaled, gmm_sales.predict(sales_data_scaled))
gmm_customer_score = silhouette_score(customer_data_scaled, gmm_customer.predict(customer_data_scaled))

# Print silhouette scores for the different clustering algorithms
print("KMeans Sales Silhouette Score:", kmeans_sales_score)
print("KMeans Customer Silhouette Score:", kmeans_customer_score)

print("DBSCAN Sales Silhouette Score:", dbscan_sales_score)
print("DBSCAN Customer Silhouette Score:", dbscan_customer_score)

print("GMM Sales Silhouette Score:", gmm_sales_score)
print("GMM Customer Silhouette Score:", gmm_customer_score)


In [ ]:


# Set up MLflow tracking URI (adjust as needed)
mlflow.set_tracking_uri("http://localhost:5300")

# Start an MLflow run to log model and metrics
mlflow.start_run()

try:
    # Log clustering model parameters
    mlflow.log_param("KMeans_clusters", 3)
    mlflow.log_param("DBSCAN_eps", 0.5)
    mlflow.log_param("DBSCAN_min_samples", 5)
    mlflow.log_param("GMM_components", 3)

    # Log model accuracy as a metric
    mlflow.log_metric("accuracy", kmeans_sales_score)

    # Log the KMeans sales model to MLflow
    mlflow.sklearn.log_model(kmeans_sales, "kmeans_sales")

    # Get the current MLflow run ID
    run_id = mlflow.active_run().info.run_id
    print(f"Run ID: {run_id}")

    # Register the model in MLflow model registry
    mlflow.register_model(f"runs:/{run_id}/kmeans_sales", "kmeans_sales")

except Exception as e:
    # Handle potential errors
    print(f"Error occurred: {e}")

finally:
    # End the MLflow run
    mlflow.end_run()

In [ ]:


# Define the model name and model instance
model_name = "Customer Data Management FOR CUST AND SALES"
model = kmeans_sales

# Initialize the MLflow client
client = MlflowClient()

# Set an MLflow experiment (create if necessary)
mlflow.set_experiment("Customer Data Management ")

# Log the model to MLflow under the model name
mlflow.sklearn.log_model(model, model_name)

# Get the active run details
run = mlflow.active_run()

# Retrieve and print the run ID
run_id = run.info.run_id
mlflow.log_metric("accuracy", kmeans_sales_score)

print(f"Run ID: {run_id}")
print(f"Model accuracy: {kmeans_sales_score}")

# Register the model in the MLflow model registry
client.create_registered_model(model_name)
client.create_model_version(model_name, f"runs:/{run_id}/random_forest_Model_test1", run_id)

# Print confirmation of successful model registration
print(f"Model {model_name} registered successfully!")
# Initialize the MLflow client
client = MlflowClient()

In [ ]:
# Assign an alias to a specific version of a registered model


# Function to assign alias to a model version
def assign_alias_to_version(model_name, version, alias):
    """
    Assign an alias to a specific version of a registered model.
    
    :param model_name: The name of the registered model.
    :param version: The version number of the model to assign the alias.
    :param alias: The alias to assign to the model version.
    :return: None
    """
     # Check if model exists
    try:
        # Set the alias for the model version
        client.set_registered_model_alias(model_name, alias, version)
        print(f"Alias '{alias}' assigned to version {version} of model '{model_name}'")
    except Exception as e:
        # Handle potential errors during alias assignment
         print(f"Error occurred: {e}")
# Parameters for assigning an alias to a specific version of a registered model
model_name = "Customer Data Management FOR CUST AND SALES"  # Name of the model to assign an alias to
version = 2  # Specify the version number to which the alias will be assigned (ensure this version exists)
alias = "currentproductionmodel"  # Specify the alias to be assigned (ensure it's a valid string)

# Call the function to assign the alias to the specified version of the model
assign_alias_to_version(model_name, version, alias)

